**Set environment**

In [1]:
source ../run_config_project.sh
show_env

You are working on             Duke Server: HARDAC
BASE DIRECTORY (FD_BASE):      /data/reddylab/Kuei
REPO DIRECTORY (FD_REPO):      /data/reddylab/Kuei/repo
WORK DIRECTORY (FD_WORK):      /data/reddylab/Kuei/work
DATA DIRECTORY (FD_DATA):      /data/reddylab/Kuei/data
CONTAINER DIR. (FD_SING):      /data/reddylab/Kuei/container

You are working with           ENCODE FCC
PATH OF PROJECT (FD_PRJ):      /data/reddylab/Kuei/repo/Proj_ENCODE_FCC
PROJECT RESULTS (FD_RES):      /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/results
PROJECT SCRIPTS (FD_EXE):      /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/scripts
PROJECT DATA    (FD_DAT):      /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/data
PROJECT NOTE    (FD_NBK):      /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/notebooks
PROJECT DOCS    (FD_DOC):      /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/docs
PROJECT LOG     (FD_LOG):      /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/log
PROJECT APP     (FD_APP):      
PROJECT REF     (FD_REF):      /data/reddylab/Kuei

## Prepare

**Check data**

In [2]:
ls ${FD_DAT}/processed/STARR_ATAC*

/data/reddylab/Kuei/repo/Proj_ENCODE_FCC/data/processed/STARR_ATAC_K562_Reddy_KS274_240311:
fragments

/data/reddylab/Kuei/repo/Proj_ENCODE_FCC/data/processed/STARR_ATAC_K562_Reddy_KS91_210401:
fragments  peaks  summary


In [3]:
FDIRY=${FD_DAT}/processed/STARR_ATAC_K562_Reddy_KS274_240311/fragments
echo ${FDIRY}
echo
ls   ${FDIRY}/*.fragments.counts.txt.gz | xargs -n 1 basename

/data/reddylab/Kuei/repo/Proj_ENCODE_FCC/data/processed/STARR_ATAC_K562_Reddy_KS274_240311/fragments

K562_ASTARR_repeat_rep1.f3q10.fragments.counts.txt.gz
K562_ASTARR_repeat_rep2.f3q10.fragments.counts.txt.gz
K562_ASTARR_repeat_rep3.f3q10.fragments.counts.txt.gz


**Test: Remove ChrM**

In [4]:
TAB="$(printf '\t')"

cat > A.bed << EOF
chr1${TAB}10${TAB}20${TAB}chr1:10-20
chr1${TAB}15${TAB}20${TAB}chr1:15-20
chrM${TAB}20${TAB}40${TAB}chrM:20-40
EOF

cat A.bed

chr1	10	20	chr1:10-20
chr1	15	20	chr1:15-20
chrM	20	40	chrM:20-40


In [5]:
cat A.bed | grep ^chrM

chrM	20	40	chrM:20-40


In [6]:
cat A.bed | grep -v ^chrM

chr1	10	20	chr1:10-20
chr1	15	20	chr1:15-20


## Execute

**Run: Loop each fragment file and output fragment counts**

In [7]:
FD_INP=${FD_DAT}/processed/STARR_ATAC_K562_Reddy_KS274_240311/fragments
FD_OUT=${FD_INP}

FN_INPS=(
    K562_ASTARR_repeat_rep1.f3q10.fragments.counts.txt.gz
    K562_ASTARR_repeat_rep2.f3q10.fragments.counts.txt.gz
    K562_ASTARR_repeat_rep3.f3q10.fragments.counts.txt.gz
)

FN_OUTS=(
    K562_ASTARR_repeat_rep1.f3q10.fragments.counts.rmChrM.txt.gz
    K562_ASTARR_repeat_rep2.f3q10.fragments.counts.rmChrM.txt.gz
    K562_ASTARR_repeat_rep3.f3q10.fragments.counts.rmChrM.txt.gz
)

In [8]:
for idx in "${!FN_INPS[@]}"; do
    FN_INP=${FN_INPS[idx]}
    FN_OUT=${FN_OUTS[idx]}
    FN_LOG=fragment.rmChrM.${FN_INP}.txt
    
    FP_INP=${FD_INP}/${FN_INP}
    FP_OUT=${FD_OUT}/${FN_OUT}
    FP_LOG=${FD_LOG}/${FN_LOG}
    
    echo "Remove mitochondria fragments..."
    echo ${FN_INP}
    echo ${FN_OUT}
    echo ${FN_LOG}
    
    sbatch -p ${NODE} \
        --exclude=dl-01   \
        --cpus-per-task 4 \
        --mem 4G \
        --output ${FP_LOG} \
        --export=FD_PRJ=${FD_PRJ},FP_INP=${FP_INP},FP_OUT=${FP_OUT} \
        <<'EOF'
#!/bin/bash
### set environment
source ${FD_PRJ}/scripts/config_project.sh

### print start message
timer_start=`date +%s`
echo "Hostname:          " $(hostname)
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}
echo "Time Stamp:        " $(date +"%m-%d-%y+%T")
echo

### show input file
echo "Input: " ${FP_INP}
echo
echo "show first few lines of input"
fun_cat ${FP_INP} | head
echo

### execute
fun_cat ${FP_INP} | grep -v ^chrM | gzip -c > ${FP_OUT}

### show output file
echo "Output: " ${FP_OUT}
echo
echo "show first few lines of output"
fun_cat ${FP_OUT} | head
echo

### print end message
timer=`date +%s`
runtime=$(echo "${timer} - ${timer_start}" | bc -l)
echo
echo 'Done!'
echo "Run Time: $(displaytime ${runtime})"
EOF
    echo
done

Remove mitochondria fragments...
K562_ASTARR_repeat_rep1.f3q10.fragments.counts.txt.gz
K562_ASTARR_repeat_rep1.f3q10.fragments.counts.rmChrM.txt.gz
fragment.rmChrM.K562_ASTARR_repeat_rep1.f3q10.fragments.counts.txt.gz.txt
Submitted batch job 30804729

Remove mitochondria fragments...
K562_ASTARR_repeat_rep2.f3q10.fragments.counts.txt.gz
K562_ASTARR_repeat_rep2.f3q10.fragments.counts.rmChrM.txt.gz
fragment.rmChrM.K562_ASTARR_repeat_rep2.f3q10.fragments.counts.txt.gz.txt
Submitted batch job 30804730

Remove mitochondria fragments...
K562_ASTARR_repeat_rep3.f3q10.fragments.counts.txt.gz
K562_ASTARR_repeat_rep3.f3q10.fragments.counts.rmChrM.txt.gz
fragment.rmChrM.K562_ASTARR_repeat_rep3.f3q10.fragments.counts.txt.gz.txt
Submitted batch job 30804731



## Review

**Check results**

In [9]:
cat ${FD_LOG}/fragment.rmChrM.K562_ASTARR_repeat_rep1.f3q10.fragments.counts.txt.gz.txt

Hostname:           x2-07-3.genome.duke.edu
Slurm Array Index: 
Time Stamp:         05-28-24+14:19:21

Input:  /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/data/processed/STARR_ATAC_K562_Reddy_KS274_240311/fragments/K562_ASTARR_repeat_rep1.f3q10.fragments.counts.txt.gz

show first few lines of input
chr1	14145	14485	chr1:14145-14485	2	.
chr1	14243	14498	chr1:14243-14498	1	.
chr1	14454	14729	chr1:14454-14729	1	.
chr1	14522	14725	chr1:14522-14725	1	.
chr1	14599	14850	chr1:14599-14850	1	.
chr1	14659	14864	chr1:14659-14864	2	.
chr1	14781	15268	chr1:14781-15268	1	.
chr1	15172	15659	chr1:15172-15659	1	.
chr1	15503	15729	chr1:15503-15729	3	.
chr1	15508	15711	chr1:15508-15711	4	.

Output:  /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/data/processed/STARR_ATAC_K562_Reddy_KS274_240311/fragments/K562_ASTARR_repeat_rep1.f3q10.fragments.counts.rmChrM.txt.gz

show first few lines of output
chr1	14145	14485	chr1:14145-14485	2	.
chr1	14243	14498	chr1:14243-14498	1	.
chr1	14454	14729	chr1:14454-14729	1	.
chr1	

In [10]:
cat ${FD_LOG}/fragment.rmChrM.K562_ASTARR_repeat_rep2.f3q10.fragments.counts.txt.gz.txt

Hostname:           x2-02-3.genome.duke.edu
Slurm Array Index: 
Time Stamp:         05-28-24+14:19:22

Input:  /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/data/processed/STARR_ATAC_K562_Reddy_KS274_240311/fragments/K562_ASTARR_repeat_rep2.f3q10.fragments.counts.txt.gz

show first few lines of input
chr1	13055	13163	chr1:13055-13163	1	.
chr1	13236	13481	chr1:13236-13481	1	.
chr1	13253	13495	chr1:13253-13495	1	.
chr1	13261	13479	chr1:13261-13479	1	.
chr1	13314	13500	chr1:13314-13500	1	.
chr1	14449	14642	chr1:14449-14642	1	.
chr1	14449	14645	chr1:14449-14645	2	.
chr1	14452	14620	chr1:14452-14620	1	.
chr1	14452	14679	chr1:14452-14679	1	.
chr1	14482	14709	chr1:14482-14709	1	.

Output:  /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/data/processed/STARR_ATAC_K562_Reddy_KS274_240311/fragments/K562_ASTARR_repeat_rep2.f3q10.fragments.counts.rmChrM.txt.gz

show first few lines of output
chr1	13055	13163	chr1:13055-13163	1	.
chr1	13236	13481	chr1:13236-13481	1	.
chr1	13253	13495	chr1:13253-13495	1	.
chr1	

In [11]:
cat ${FD_LOG}/fragment.rmChrM.K562_ASTARR_repeat_rep3.f3q10.fragments.counts.txt.gz.txt

Hostname:           x2-03-2.genome.duke.edu
Slurm Array Index: 
Time Stamp:         05-28-24+14:19:23

Input:  /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/data/processed/STARR_ATAC_K562_Reddy_KS274_240311/fragments/K562_ASTARR_repeat_rep3.f3q10.fragments.counts.txt.gz

show first few lines of input
chr1	10421	10570	chr1:10421-10570	1	.
chr1	13044	13243	chr1:13044-13243	2	.
chr1	13303	13459	chr1:13303-13459	1	.
chr1	13483	13661	chr1:13483-13661	1	.
chr1	14091	14513	chr1:14091-14513	1	.
chr1	14129	14468	chr1:14129-14468	1	.
chr1	14191	14755	chr1:14191-14755	1	.
chr1	14231	14498	chr1:14231-14498	1	.
chr1	14449	14971	chr1:14449-14971	1	.
chr1	14452	14620	chr1:14452-14620	1	.

Output:  /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/data/processed/STARR_ATAC_K562_Reddy_KS274_240311/fragments/K562_ASTARR_repeat_rep3.f3q10.fragments.counts.rmChrM.txt.gz

show first few lines of output
chr1	10421	10570	chr1:10421-10570	1	.
chr1	13044	13243	chr1:13044-13243	2	.
chr1	13303	13459	chr1:13303-13459	1	.
chr1	